<a href="https://colab.research.google.com/github/thanksbinh/minBert/blob/main/minBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
  

In [3]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
ds

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [5]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [6]:
# # read it in to inspect it
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# print("length of dataset in characters: ", len(text))
# # let's look at the first 1000 characters
# print(text[:1000])

In [72]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
# batch_size = 64 # how many independent sequences will we process in parallel?
# block_size = 256 # what is the maximum context length for predictions?
batch_size = 4
block_size = 8
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 64
n_heads = 4 # --> every head is 256/4 = 64 dimensional
n_layers = 4
# dropout = 0.2
dropout = 0.0
max_sentences = 10000
# ------------

In [73]:
# prompt: write stoi and itos functions with google-bert/bert-base-cased

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-cased')
tokenizer.add_tokens(['\n'])


def stoi(text):
  """
  String to index.
  Converts a string to a list of token indices using the BERT tokenizer.
  """
  inputs = tokenizer(text, return_tensors="pt").to(device)
  return inputs.input_ids[0]


def itos(token_indices):
  """
  Index to string.
  Converts a list of token indices to a string using the BERT tokenizer.
  """
  # tokens = tokenizer.convert_ids_to_tokens(token_indices)
  # return tokenizer.convert_tokens_to_string(tokens, clean_up_tokenization_spaces=True)
  return tokenizer.decode(token_indices, clean_up_tokenization_spaces=True, skip_special_tokens=True)

def encode(s):
  # return stoi(s)
  sentences = s.split('.')
  encoded_sentences = []
  for sentence in sentences:
    encoded_sentences.append(stoi(sentence))
  # return encoded_sentences
  return torch.cat(encoded_sentences)

def decode(l):
  return itos(l)
  # decoded_sentences = []
  # for sentence in l:
  #   decoded_sentences.append(itos(sentence))
  # return '.'.join(decoded_sentences)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [74]:
ds[1]['text']

' = Valkyria Chronicles III = \n'

In [75]:
# create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode(ds[0]['text']))
print(decode(encode(ds[0]['text'])))
print(ds[0]['text'] == '')

tensor([101, 102])

True


In [76]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
# data = torch.tensor(encode(text), dtype=torch.long)
data = []
# for i in range(len(ds)):
for i in range(max_sentences):
  if ds[i]['text'] == '':
    continue
  data.append(torch.tensor(encode(ds[i]['text']), dtype=torch.long))
data = torch.cat(data)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

<ipython-input-76-82caababbde6>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.append(torch.tensor(encode(ds[i]['text']), dtype=torch.long))


torch.Size([683072]) torch.int64
tensor([  101,   134, 12226,  3781,  3464, 17758,  2684,   134, 28996,   102,
          101, 14895, 21006,  1185, 12226,  3781,  3464,   124,   131,   133,
         8362,  1377,   135, 17758,   113,  1983,   131,   100,   100,   100,
          117,  4941,   102,   101, 12226,  3781,  3464,  1104,  1103,  2651,
         2427,   124,   114,   117,  3337,  2752,  1106,  1112, 12226,  3781,
         3464, 17758,  2684,  1796,  1999,   117,  1110,   170, 12394,  1648,
          137,   118,   137,  1773,  1888,  1342,  1872,  1118, 17979,  1105,
         3957,   102,   101, 11724,  1111,  1103,  8905,  3905,  1895,   102,
          101, 11930,  1107,  1356,  1349,  1107,  1999,   117,  1122,  1110,
         1103,  1503,  1342,  1107,  1103, 12226,  3781,  3464,  1326,   102,
          101, 18653,  1643, 26179,  1158,  1103,  1269, 11970,  1104, 12394,
         1105,  1842,   137,   118,   137,  1159, 11716,  1112,  1157, 17543,
          117,  1103,  1642,  2

In [77]:
vocab_size = max(data) + 1
# chars = sorted(list(set(data.tolist())))
# print(chars)
# vocab_size = len(chars)
print(vocab_size)

tensor(28997)


In [78]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n]
val_data = data[n:]

In [79]:
train_data[:block_size+1]

tensor([  101,   134, 12226,  3781,  3464, 17758,  2684,   134, 28996])

In [80]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f"when input is {context} the target: {target}")

In [81]:
import random
block_size_temp = 8

x = train_data[:block_size_temp]
y = train_data[:block_size_temp]
context = torch.tensor([103 if random.uniform(0, 1) < 0.15 else item.item() for item in x])
target = y
print(f"when input is {context} the target: {target}")

when input is tensor([  101,   134, 12226,   103,  3464, 17758,  2684,   134]) the target: tensor([  101,   134, 12226,  3781,  3464, 17758,  2684,   134])


In [82]:
from torch.nn import CrossEntropyLoss

def mask_tokens(inputs, mlm_probability=0.15):
    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, mlm_probability)

    # Create a mask where 15% of tokens will be masked
    mask = torch.bernoulli(probability_matrix).bool()

    # Make sure we don’t mask special tokens like [CLS], [SEP]
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    mask = mask & ~torch.tensor(special_tokens_mask, dtype=torch.bool)

    # 80% of the time, replace masked input tokens with [MASK]
    inputs[mask] = tokenizer.convert_tokens_to_ids("[MASK]")


    # 10% of the time, replace masked input tokens with random words
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    ran = torch.rand(labels.shape) < 0.1
    inputs[mask & ran] = random_words[mask & ran]

    # 10% of the time, keep the original word (no change)

    # Return masked inputs and the labels (for calculating the loss)
    return inputs, labels

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    batch_data = torch.stack([data[i:i + block_size] for i in ix], dim=0)
    inputs, labels = mask_tokens(batch_data)

    inputs, labels = inputs.to(device), labels.to(device)
    return inputs, labels

def get_batch_gpt(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            inputs, labels = get_batch(split)
            logits = model(inputs)

            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
            losses[k] = loss.item()

        out[split] = losses.mean()
    model.train()
    return out

inputs, labels = get_batch('train')
print('inputs:')
print(inputs.shape)
print(inputs)
print('labels:')
print(labels.shape)
print(labels)

# print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1194,  1103,  1171,   103,  1104,   103, 27774,  1110],
        [ 1527,   103,  6824,   113,   177,  2101,  1161, 15921],
        [  103,  1107,  1103, 12120,  1643, 27089,  6834,   103],
        [17919, 24593,  3555,  1115,  1112,   103,  1112,   124]])
labels:
torch.Size([4, 8])
tensor([[ 1194,  1103,  1171,  1442,  1104,  1103, 27774,  1110],
        [ 1527,   182,  6824,   113,   177,  2101,  1161,   132],
        [18142,  1107,  1103, 12120,  1643, 27089,  6834,  1777],
        [17919, 24593,  3555,  1115,  1112,  1242,  1112,   124]])


In [83]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # compute attention score "affinities"
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

        # GPT
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # Todo: check this part again
        # attention_mask = ...
        # wei = wei.masked_fill(attention_mask == 0, float('-inf'))  # Mask padding tokens

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) # Prevent some nodes from communicating
        out = wei @ v
        return out

In [84]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [85]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [86]:
class EncoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        # Residual connection, clean flow
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [87]:
class DecoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [88]:
class Bert(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[EncoderBlock(n_embed, n_heads=n_heads) for _ in range(n_layers)])
        # self.blocks = nn.Sequential(*[DecoderBlock(n_embed, n_heads=n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        # self.lm_head = nn.Linear(n_embed, vocab_size)
        self.cls = nn.Linear(n_embed, vocab_size) # for MLM
        self.last_hidden_state = None

    def forward(self, idx, attention_mask=None, token_type_ids=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        token_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = token_embed + pos_embed # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        # logits = self.lm_head(x) # (B,T,vocab_size)
        logits = self.cls(x)  # (B,T,vocab_size)

        # if targets is None:
        #     loss = None
        # else:
        #     B, T, C = logits.shape
        #     logits = logits.view(B*T, C)
        #     targets = targets.view(B*T)

        #     loss = F.cross_entropy(logits, targets)

        self.last_hidden_state = x
        return logits

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size token
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = Bert()
m = model.to(device)

In [89]:
# xb, yb = get_batch_gpt('train')

# logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)

In [90]:
inputs, labels = get_batch('train')

logits = m(inputs)
print(logits.shape)

# Assume `prediction_logits` are the outputs from your model, and `labels` are the true token IDs
loss_fct = CrossEntropyLoss()
masked_lm_loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
print(masked_lm_loss)

torch.Size([4, 8, 28997])


In [93]:
# print(decode(m.generate(idx = torch.tensor([[101]], dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

In [94]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [97]:
max_iters = 5000
eval_interval = 500

for iter in range(max_iters + 1):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    inputs, labels = get_batch('train')

    # evaluate the loss
    logits = m(inputs)
    loss = F.cross_entropy(logits.view(-1, vocab_size), labels.view(-1))

    # zero the gradients from the previous iteration
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.1292, val loss 4.2572
step 500: train loss 3.3660, val loss 3.8265
step 1000: train loss 3.0755, val loss 3.2719
step 1500: train loss 2.8869, val loss 3.0714
step 2000: train loss 2.5181, val loss 2.8869
step 2500: train loss 2.3417, val loss 2.7723
step 3000: train loss 2.3129, val loss 2.6881
step 3500: train loss 2.0759, val loss 2.6035
step 4000: train loss 1.9396, val loss 2.4473
step 4500: train loss 2.0077, val loss 2.5031
step 5000: train loss 2.0034, val loss 2.3348


In [98]:
import pickle

In [99]:
filename = 'transformer.sav'
pickle.dump(m, open(filename, 'wb'))

In [100]:
# loading the saved model
loaded_model = pickle.load(open('transformer.sav', 'rb'))

In [103]:
# print(decode(loaded_model.generate(idx = torch.tensor([[ 101 ]], dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))

In [164]:
# sample a batch of data
inputs, labels = get_batch('train')
inputs = inputs.clone()
inputs[inputs == 103] = 168
print((inputs[0]))
print((labels[0]))
print(decode(inputs[0]))
print(decode(labels[0]))

# evaluate the loss
logits = m(inputs)
print(logits.shape, logits)

tensor([1118, 5227,  117, 4787,  168, 2468, 1106, 1115])
tensor([1118, 5227,  117, 4787, 1117, 2468, 1106, 1115])
by DC, changing _ appearance to that
by DC, changing his appearance to that
torch.Size([4, 8, 28997]) tensor([[[-4.0308, -3.8404, -3.7642,  ..., -4.0805, -3.8901, -2.6055],
         [-1.0732, -0.8171, -0.7572,  ..., -1.0807, -0.8685,  1.6793],
         [-5.2601, -5.1246, -4.9994,  ..., -5.2172, -4.9934, -1.6572],
         ...,
         [ 0.4561,  0.6391,  0.6893,  ...,  0.3674,  0.5943, -1.6085],
         [-7.8535, -7.5912, -7.4743,  ..., -7.7101, -7.5470,  3.5356],
         [-4.5251, -4.2866, -4.2148,  ..., -4.4660, -4.2847, -2.1392]],

        [[-8.0250, -7.8147, -7.6957,  ..., -7.8904, -7.6783,  4.5519],
         [-4.6238, -4.4220, -4.3465,  ..., -4.4834, -4.3581, -2.1406],
         [ 0.7844,  1.0764,  1.1160,  ...,  0.7885,  1.0011,  0.6642],
         ...,
         [-2.1058, -1.8793, -1.8319,  ..., -2.1285, -1.9853, -2.6777],
         [-5.3248, -5.0455, -4.9541,  ..., -

In [165]:
logits = logits[:, 4, :] # becomes (B, C)
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (B, C)
# sample from the distribution
print('probs', probs.shape, probs)
idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
# # append sampled index to the running sequence
print('idx_next', idx_next)

probs torch.Size([4, 28997]) tensor([[1.7665e-06, 2.1028e-06, 2.2395e-06,  ..., 1.6564e-06, 1.9720e-06,
         2.3587e-07],
        [1.1940e-07, 1.6417e-07, 1.6845e-07,  ..., 1.1900e-07, 1.5153e-07,
         2.1936e-10],
        [4.2343e-07, 5.3662e-07, 5.5930e-07,  ..., 4.2312e-07, 5.2085e-07,
         2.0066e-06],
        [1.9749e-06, 2.4177e-06, 2.5628e-06,  ..., 1.8837e-06, 2.2077e-06,
         3.1463e-07]], grad_fn=<SoftmaxBackward0>)
idx_next tensor([[5904],
        [4162],
        [1339],
        [2617]])


In [166]:
decoded_sentences = []

for idx in idx_next:
    decoded_sentences.append(itos(idx.tolist()))

print(decoded_sentences)

['boundary', 'storm', 'face', 'Bill']


In [104]:
class BERTForQuestionAnswering(nn.Module):
    def __init__(self):
        super(BERTForQuestionAnswering, self).__init__()

        # Load the BERT model
        self.bert = loaded_model

        # Define the start position head
        self.start_head = nn.Linear(n_embed, 1)

        # Define the end position head
        self.end_head = nn.Linear(n_embed, 1)

        # Optional: Initialize weights
        nn.init.normal_(self.start_head.weight, std=0.02)
        nn.init.normal_(self.end_head.weight, std=0.02)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Get BERT outputs
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # Get the last hidden state (shape: [batch_size, sequence_length, hidden_size])
        sequence_output = outputs.last_hidden_state

        # Apply start head (shape: [batch_size, sequence_length, 1])
        start_logits = self.start_head(sequence_output).squeeze(-1)

        # Apply end head (shape: [batch_size, sequence_length, 1])
        end_logits = self.end_head(sequence_output).squeeze(-1)

        return start_logits, end_logits


model_qa = BERTForQuestionAnswering()
m_qa = model_qa.to(device)

In [ ]:
inputs, labels = get_batch('train')

start_logits, end_logits = m_qa(inputs)
print(start_logits.shape, end_logits.shape)

# loss_fct = CrossEntropyLoss()
# loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
# print(loss)